In [10]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

from data_loader import data_label_split
from data_loader import generate_data_set
from data_loader import dmso_taxol_ProfileBag
from torch_exp import mini_noise_signal_cv

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier

import torch
import torch.utils.data as D 
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim

In [1]:

drop_NA_data = pd.read_csv("moa_data_drop_NA.csv", index_col=0)

In [9]:
mini_data = pd.read_csv("mini_moa_data_drop_NA.csv",index_col=0)

In [3]:
drop_NA_data['compound'].value_counts()

DMSO     244072
taxol     56763
Name: compound, dtype: int64

In [5]:
# takes the df with "compound" columns
class dmso_taxol_ProfileBag(D.Dataset):
    def __init__(self, df, size, bag_mean_size, bag_std_size, perc, treatment, control, merged_perc):
        self.df = df
        self.size = size
        merged_perc = 0.5
        self.merged_pos_size = int(size * merged_perc)
        self.unmerged_neg_size = int(size * (1 - merged_perc))
        
        self.bag_mean_size = bag_mean_size
        self.bag_std_size = bag_std_size
        self.perc = perc
        self.treatment = treatment
        self.control = control
        self.bag_size = np.random.normal(bag_mean_size, bag_std_size, size).astype(int)


        self.treatment_data = self.df[self.df["compound"] == self.treatment]
        self.control_data = self.df[self.df["compound"] == self.control]
        self.treatment_size = (self.bag_size * self.perc).astype(int)
        self.control_size = self.bag_size - self.treatment_size
        
    def _get_sampling_index(self):
        treat_index_list = []
        control_index_list = []

        treat_length = len(self.treatment_data)
        control_length = len(self.control_data)
        
        index = np.arange(treat_length)
        for i in range(self.merged_pos_size):
            if len(index) < self.treatment_size[i]:
                index = np.arange(treat_length)
            z = np.random.choice(index, self.treatment_size[i], replace=False)
            index = index[~np.isin(index, z)]
            treat_index_list.append(z)

        index = np.arange(control_length)
        for i in range(self.merged_pos_size):
            if len(index) < self.control_size[i]:
                index = np.arange(control_length)
            z = np.random.choice(index, self.control_size[i], replace=False)
            index = index[~np.isin(index, z)]
            control_index_list.append(z)
        return treat_index_list, control_index_list
        
        
    def __getitem__(self, index):
        if index < self.merged_pos_size:
            treat_index_list, control_index_list= self._get_sampling_index()
            merged_data = self.treatment_data.iloc[treat_index_list[index]].append(self.control_data.iloc[control_index_list[index]]) 
            X, y = data_label_split(merged_data.sample(frac=1))
            return torch.from_numpy(X.values), [torch.tensor([1.0]), list(y["compound"])]
        else:
            unmerged_bag_size = np.random.normal(self.bag_mean_size, self.bag_std_size, 1).astype(int).item()
            X, y = data_label_split(self.control_data.sample(unmerged_bag_size))
            return torch.from_numpy(X.values), [torch.tensor([0.0]), list(y["compound"])]
        
    def __len__(self):
        return self.size

In [13]:
sf_drop_NA_data = mini_data[["compound", "concentration",
                                "moa", "row ID", "Iteration (#2)", "COND",
                               "AreaShape_Area_Nuclei", "AreaShape_Compactness_Nuclei"]]
bagData = dmso_taxol_ProfileBag(sf_drop_NA_data, 200, 50, 5, 0.5, "taxol", "DMSO", 0.5)
dataloader = D.DataLoader(bagData, batch_size=1, shuffle=True)

In [238]:
data, label = iter(dataloader).next()

# Deep Set Model

In [14]:
class SmallDeepSet(nn.Module):
    def __init__(self, pool="max"):
        super().__init__()
        self.enc = nn.Sequential(
            nn.Linear(in_features=2, out_features=16),
            nn.ReLU(),
            nn.Linear(in_features=16, out_features=32),
            nn.ReLU(),
            nn.Linear(in_features=32, out_features=64),
        )
        self.dec = nn.Sequential(
            nn.Linear(in_features=64, out_features=32),
            nn.ReLU(),
            nn.Linear(in_features=32, out_features=1),
            nn.Sigmoid()
        )
        self.pool = pool

    def forward(self, x):
        x = self.enc(x)
        if self.pool == "max":
            x = x.max(dim=1)[0]
        elif self.pool == "mean":
            x = x.mean(dim=1)
        elif self.pool == "sum":
            x = x.sum(dim=1)
        elif self.pool == "min":
            x = x.min(dim=1)[0]
        x = self.dec(x)
        return x, torch.ge(x, 0.5)

In [183]:
def train(epoch, loader, model, opt):
    model.train()
    train_loss = 0.
    train_error = 0.
    for batch_idx, (data, label) in tqdm(enumerate(loader), desc = 'For %d epoch' % epoch): 
        bag_label = label[0]
        data, bag_label = data.float().cuda(), bag_label.cuda()
        data, bag_label = Variable(data), Variable(bag_label)

        # reset gradients
        optimizer.zero_grad()
        # calculate loss and metrics
        y_prob, y_hat = model(data)
        loss = nn.BCELoss()(y_prob, bag_label)
        train_loss += loss.data.cpu()
        error = 1-y_hat.float().eq(bag_label).cpu().float().mean().item()
        train_error += error
        # backward pass
        loss.backward()
        # step
        optimizer.step()

    # calculate mean loss and error for epoch
    train_loss /= len(loader)
    train_error /= len(loader)
    return(train_loss, train_error)
    print('Epoch: {}, Loss: {:.4f}, Train error: {:.4f}'.format(epoch, train_loss.item(), train_error))




In [184]:
def test(model, loader):
    model.eval()
    pred_score_control = []
    pred_score_treat = []
    acc_control = []
    acc_treat = []
    
    for batch_idx, (data, label) in enumerate(loader): 
        bag_label = label[0]
        data, bag_label = data.float().cuda(), bag_label.cuda()
        y_prob, y_hat = model(data)

        if int(bag_label.item()) == 0:
            pred_score_control.append(y_prob.cpu().float().item())
            acc_control.append(y_hat.float().eq(bag_label).cpu().float().mean().item())
        else:
            pred_score_treat.append(y_prob.cpu().float().item())
            acc_treat.append(y_hat.float().eq(bag_label).cpu().float().mean().item())
    return acc_control, acc_treat, pred_score_control, pred_score_treat

In [257]:
def mini_noise_signal_cv(data, num_bag, bag_size_mean, bag_size_std, treatment, control, model, splits, epochs):
    mean_control_accuracy=[]
    std_control_accuracy=[]
    mean_treat_accuracy=[]
    std_treat_accuracy=[]
    mean_pred_score_control = []
    std_pred_score_control = []
    mean_pred_score_treatment = []
    std_pred_score_treatment = []

    
    # Set different percentage of treatment v.s. control 
    for j in tqdm(range(5,96,5), desc = "training at different percent"):
        X, y = data_label_split(data)
        y = y["compound"]
        
        acc_control_list = []
        acc_treat_list = []
        pred_score_control_list = []
        pred_score_treat_list = []
        # Stratified K fold 
        skf = StratifiedKFold(n_splits = splits)
        for i, (train_index, test_index) in tqdm(enumerate(skf.split(X, y)), desc="%d fold cross validation"%splits):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]  
            X_train = pd.concat([X_train, y_train], axis=1, sort=False)
            X_test = pd.concat([X_test, y_test], axis=1, sort=False)
            
            # Redefine dataloader and train model at each fold
            train_dataset = dmso_taxol_ProfileBag(X_train, num_bag, bag_size_mean, bag_size_std, j/100, treatment, control, 0.5)
            valida_dataset = dmso_taxol_ProfileBag(X_test, num_bag, bag_size_mean, bag_size_std, j/100, treatment, control, 0.5)
            train_loader = D.DataLoader(train_dataset, batch_size=1, shuffle=True)
            valida_loader = D.DataLoader(valida_dataset, batch_size=1, shuffle=True)
            
            train_loss, train_error = train(epochs, train_loader, model, optimizer)
            acc_control, acc_treat, pred_score_control, pred_score_treat = test(model, valida_loader)
            
            acc_control_list+=acc_control
            acc_treat_list+=acc_treat
            pred_score_control_list+=pred_score_control
            pred_score_treat_list+=pred_score_treat
            
        mean_control_accuracy.append(np.mean(acc_control_list))
        std_control_accuracy.append(np.std(acc_control_list))
        mean_treat_accuracy.append(np.mean(acc_treat_list))
        std_treat_accuracy.append(np.std(acc_treat_list))
        
        mean_pred_score_control.append(np.mean(pred_score_control_list))
        std_pred_score_control.append(np.std(pred_score_control_list))
        mean_pred_score_treatment.append(np.mean(pred_score_treat_list))
        std_pred_score_treatment.append(np.std(pred_score_treat_list))
    return (mean_control_accuracy, std_control_accuracy, 
            mean_treat_accuracy, std_treat_accuracy, 
            mean_pred_score_control, std_pred_score_control,
            mean_pred_score_treatment, std_pred_score_treatment)



In [151]:
full_deepset = SmallDeepSet().cuda()

In [7]:
sf_deepset = SmallDeepSet().cuda()

In [9]:
optimizer = optim.Adam(sf_deepset.parameters(), lr=0.0005, betas=(0.9, 0.999), weight_decay=10e-5)

In [ ]:
results = mini_noise_signal_cv(drop_NA_data, 1000, 100, 10, "taxol", "DMSO", full_deepset, 10, 50)

  0%|          | 0/19 [00:00<?, ?it/s]
0it [00:00, ?it/s]/root/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/root/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


0it [00:00, ?it/s]

1it [00:05,  5.63s/it]

2it [00:11,  5.64s/it]

3it [00:16,  5.60s/it]

6it [00:22,  4.47s/it]

7it [00:27,  4.78s/it]

9it [00:33,  4.17s/it]

10it [00:38,  4

In [12]:
sf_results = mini_noise_signal_cv(sf_drop_NA_data, 10, 100, 10, "taxol", "DMSO", sf_deepset, optimizer,2, 1)

training at different percent:   0%|          | 0/19 [00:00<?, ?it/s]
2 fold cross validation: 0it [00:00, ?it/s]

For 1 epoch: 0it [00:00, ?it/s]

For 1 epoch: 1it [00:00,  3.74it/s]

For 1 epoch: 10it [00:00, 24.40it/s][A

2 fold cross validation: 1it [00:00,  1.91it/s]

For 1 epoch: 0it [00:00, ?it/s]

For 1 epoch: 10it [00:00, 69.69it/s][A

2 fold cross validation: 2it [00:00,  2.58it/s]
training at different percent:   5%|▌         | 1/19 [00:00<00:14,  1.27it/s]
2 fold cross validation: 0it [00:00, ?it/s]

For 1 epoch: 0it [00:00, ?it/s]

For 1 epoch: 10it [00:00, 72.66it/s][A

2 fold cross validation: 1it [00:00,  3.83it/s]

For 1 epoch: 0it [00:00, ?it/s]

For 1 epoch: 10it [00:00, 72.10it/s][A

2 fold cross validation: 2it [00:00,  3.95it/s]
training at different percent:  11%|█         | 2/19 [00:01<00:11,  1.42it/s]
2 fold cross validation: 0it [00:00, ?it/s]

For 1 epoch: 0it [00:00, ?it/s]

For 1 epoch: 10it [00:00, 92.59it/s]

2 fold cross validation: 1it [00:00,  5.12it/

In [ ]:
sf_results

In [246]:
acc_control, acc_treat, pred_score_control, pred_score_treat = test(sf_deepset, dataloader)

In [253]:
(mean_control_accuracy, std_control_accuracy, 
            mean_treat_accuracy, std_treat_accuracy, 
            mean_pred_score_control, std_pred_score_control,
            mean_pred_score_treatment, std_pred_score_treatment) = sf_results

In [23]:
sf_results[0]

[0.6,
 0.1,
 0.4,
 0.9,
 0.9,
 0.7,
 0.8,
 1.0,
 0.6,
 0.7,
 0.8,
 0.6,
 0.8,
 1.0,
 1.0,
 0.7,
 0.7,
 1.0,
 0.8]

In [ ]:
dict = {'control_accuracy': nme, 'std_control_accuracy': deg, 'score': scr}  

In [24]:
sf_results = pd.DataFrame(list(sf_results)).transpose()

In [25]:
sf_results.columns = ["mean_control_accuracy", "std_control_accuracy", 
            "mean_treat_accuracy", "std_treat_accuracy", 
            "mean_pred_score_control", "std_pred_score_control",
            "mean_pred_score_treatment", "std_pred_score_treatment"]

In [26]:
sf_results

,mean_control_accuracy,std_control_accuracy,mean_treat_accuracy,std_treat_accuracy,mean_pred_score_control,std_pred_score_control,mean_pred_score_treatment,std_pred_score_treatment
0,0.6,0.489898,0.4,0.489898,0.392564,0.385677,0.393943,0.422387
1,0.1,0.300000,1.0,0.000000,0.722624,0.191698,0.814537,0.104709
2,0.4,0.489898,0.7,0.458258,0.543045,0.198318,0.673363,0.171835
3,0.9,0.300000,0.3,0.458258,0.213512,0.131968,0.393299,0.203889
4,0.9,0.300000,0.3,0.458258,0.187630,0.155302,0.295522,0.244676
5,0.7,0.458258,0.6,0.489898,0.324578,0.361401,0.679215,0.287370
6,0.8,0.400000,0.9,0.300000,0.343598,0.340132,0.728700,0.192789
7,1.0,0.000000,0.8,0.400000,0.134028,0.126734,0.733124,0.210020
8,0.6,0.489898,0.7,0.458258,0.386999,0.428574,0.691723,0.349524
9,0.7,0.458258,1.0,0.000000,0.397993,0.306416,0.989802,0.014643
